In [7]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
# Download NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
df=pd.read_csv("1429_1.csv")

/tmp/ipython-input-450841150.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("1429_1.csv")


In [10]:
print(df.head())

                     id                                               name  \
0  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
1  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
2  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
3  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
4  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   

        asins   brand                                         categories  \
0  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
1  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
2  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
3  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
4  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   

                                                keys manufacturer  \
0  84

In [11]:
df.shape

(34660, 21)

In [32]:


df=df[(df['categories'] == 'Back To College,College Electronics,College Tvs & Home Theater,Electronics,Tvs & Home Theater,Streaming Devices,Featured Brands,Amazon Devices,Holiday Shop,Ways To Shop,TV & Home Theater,Streaming Media Players,All Streaming Media Players,TVs Entertainment,Video Games,Kindle Store,Electronics Features,Kids & Family,Fire TV') & (df['reviews.rating'] == 5)]

print(df.shape)

(0, 21)


In [13]:
print(df.columns.tolist())



['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username']


In [33]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load your dataset
# df = df['reviews.text']  # must have a 'review' column
all_reviews_text = " ".join(df['reviews.text'].dropna().astype(str))
# Load DistilBERT-based embedding model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

def extractive_summary(text, top_n=5):
    sentences = text.split(". ")
    sentences = [s.strip() for s in sentences if s]
    embeddings = model.encode(sentences)
    doc_emb = np.mean(embeddings, axis=0)
    similarities = [util.cos_sim(doc_emb, emb)[0][0].item() for emb in embeddings]
    top_idx = np.argsort(similarities)[-top_n:]
    return ". ".join([sentences[i] for i in sorted(top_idx)])

# Apply summarization
summarize_text= extractive_summary(all_reviews_text, top_n=5)
print(summarize_text)

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [15]:
from transformers import pipeline
summarizer = pipeline("summarization", model="google/flan-t5-large")
all_reviews_text = " ".join(df['reviews.text'].dropna().astype(str))
summary = summarizer(
    "summarize: " + all_reviews_text,
    max_length=150,
    min_length=40,
    do_sample=False
)
print(summary[0]['summary_text'])


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (2705 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


I purchased this firetv to upgrade my from first firetv version as I have a samsung 4k tv, this new firetv recognized my hdmi port as 4k compatible when I set the resolution to 4k on video settings, after trying all posible settings, this bug only gives 1080 even playing 4k content.


In [16]:
print(summary)

[{'summary_text': 'I purchased this firetv to upgrade my from first firetv version as I have a samsung 4k tv, this new firetv recognized my hdmi port as 4k compatible when I set the resolution to 4k on video settings, after trying all posible settings, this bug only gives 1080 even playing 4k content.'}]


In [20]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import numpy as np
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# -------------------------------
# Step 1: Extractive Summarization
# -------------------------------
text = " ".join(df['reviews.text'].dropna().astype(str))

# Split into sentences
sentences = sent_tokenize(text)



# Get sentence embeddings
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')
sentence_embeddings = embedder.encode(sentences, convert_to_tensor=True)

# Compute similarity with the "document centroid"
# document_embedding = np.mean(sentence_embeddings.cpu().numpy(), axis=0)
document_embedding = sentence_embeddings.mean(dim=0, keepdim=True)

cosine_scores = util.cos_sim(sentence_embeddings, document_embedding)

# Pick top-N most representative sentences
top_n = 5
top_sentence_indices = np.argsort(-cosine_scores.squeeze().cpu().numpy())[:top_n]
selected_sentences = [sentences[i] for i in sorted(top_sentence_indices)]

extractive_summary = " ".join(selected_sentences)
print("Extractive Summary:\n", extractive_summary)

# -------------------------------
# Step 2: Abstractive Summarization
# -------------------------------
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

final_summary = summarizer(extractive_summary, max_length=120, min_length=40, do_sample=False)
print("\nFinal Abstractive Summary:\n", final_summary[0]['summary_text'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Extractive Summary:
 Since neither of the above worked, this made the device completely redundant with my Chromecast. It locked up on installing kodi and it wouldn't recover. When the remote works, the Amazon TV seems sluggish in comparison to the other products I tried. When I returned a few hours later it was stuck in a perpetual boot and was not recoverable. It has completely stopped responding to the remote.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Your max_length is set to 120, but your input_length is only 81. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)



Final Abstractive Summary:
 When the remote works, the Amazon TV seems sluggish in comparison to the other products I tried. When I returned a few hours later it was stuck in a perpetual boot and was not recoverable. Since neither of the above worked, this made the device completely redundant with my Chromecast.


In [37]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load tokenizer & model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [38]:
import pandas as pd
from datasets import Dataset

data = {
    "reviews.text": [
        "The phone is great but the battery drains quickly.",
        "Amazing camera quality and display. Totally worth it!",
        "Software crashes often. Not recommended."
    ],
    "summary": [
        "Good phone, poor battery.",
        "Great camera and display.",
        "Buggy software, not recommended."
    ]
}

df_set = pd.DataFrame(data)
dataset = Dataset.from_pandas(df_set)


In [39]:
print(dataset.shape)

(3, 2)


In [40]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["reviews.text"],
        max_length=512,
        truncation=True
    )
    labels = tokenizer(
        examples["summary"],
        max_length=64,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [41]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./review_summarizer",
    # evaluation_strategy="no",      # change to "epoch" for larger data
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
)


In [42]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,  # ✅ this fixes the ValueError
)

trainer.train()


Step,Training Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2, training_loss=2.5759899616241455, metrics={'train_runtime': 54.7244, 'train_samples_per_second': 0.055, 'train_steps_per_second': 0.037, 'total_flos': 76187271168.0, 'train_loss': 2.5759899616241455, 'epoch': 1.0})

In [54]:
import torch
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [44]:
model.save_pretrained("./fine_tuned_bart_review_summarizer")
tokenizer.save_pretrained("./fine_tuned_bart_review_summarizer")


('./fine_tuned_bart_review_summarizer/tokenizer_config.json',
 './fine_tuned_bart_review_summarizer/special_tokens_map.json',
 './fine_tuned_bart_review_summarizer/vocab.json',
 './fine_tuned_bart_review_summarizer/merges.txt',
 './fine_tuned_bart_review_summarizer/added_tokens.json')

In [60]:
model = BartForConditionalGeneration.from_pretrained("./fine_tuned_bart_review_summarizer").to(device)
tokenizer = BartTokenizer.from_pretrained("./fine_tuned_bart_review_summarizer")


/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [66]:
all_reviews_text =  " ".join(df['reviews.text'].dropna().astype(str))

inputs = tokenizer([all_reviews_text], max_length=512, return_tensors="pt", truncation=True)
summary_ids = model.generate(inputs["input_ids"].to(device), max_length=60, min_length=10, num_beams=4, early_stopping=True, length_penalty=1.0)
print("Generated Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


Generated Summary: CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.


In [68]:
!zip -r fine_tuned_bart_review_summarizer.zip fine_tuned_bart_review_summarizer


  adding: fine_tuned_bart_review_summarizer/ (stored 0%)
  adding: fine_tuned_bart_review_summarizer/generation_config.json (deflated 47%)
  adding: fine_tuned_bart_review_summarizer/vocab.txt (deflated 53%)
  adding: fine_tuned_bart_review_summarizer/README.md (deflated 59%)
  adding: fine_tuned_bart_review_summarizer/model.safetensors (deflated 8%)
  adding: fine_tuned_bart_review_summarizer/1_Pooling/ (stored 0%)
  adding: fine_tuned_bart_review_summarizer/1_Pooling/config.json (deflated 59%)
  adding: fine_tuned_bart_review_summarizer/sentence_bert_config.json (deflated 9%)
  adding: fine_tuned_bart_review_summarizer/merges.txt (deflated 53%)
  adding: fine_tuned_bart_review_summarizer/tokenizer.json (deflated 71%)
  adding: fine_tuned_bart_review_summarizer/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_bart_review_summarizer/config_sentence_transformers.json (deflated 40%)
  adding: fine_tuned_bart_review_summarizer/config.json (deflated 62%)
  adding: fine_tuned_bart_